In [1]:
# Importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pivoting

In [2]:
# Importing the .csv saved at the end of STEP 1 (with cleaned CUSIPs and sectors + unnecessary fields removed)
refinitiv = pd.read_csv('CleanedNonPivot_1_30_25.csv', header=0)

In [3]:
# Pivot the data so that each fieldname is a separate column
df = refinitiv.copy()
pivot = pd.pivot(df, values=['value'], index=['year', 'cusip', 'comname', 'sector'], columns=['fieldname'])
pivot = pivot.reset_index()
print(pivot)

           year     cusip                                 comname  \
fieldname                                                           
0          2002  00105510                      AFLAC INCORPORATED   
1          2002  00184A10                        TIME WARNER INC.   
2          2002  00185310                        Clearfield, Inc.   
3          2002  00195750                              AT&T CORP.   
4          2002  00282410                     ABBOTT LABORATORIES   
...         ...       ...                                     ...   
40313      2023  98980G10                           ZSCALER, INC.   
40314      2023  98980L10               ZOOM COMMUNICATIONS, INC.   
40315      2023  98981710                             ZUMIEZ INC.   
40316      2023  98983L10  ZURN ELKAY WATER SOLUTIONS CORPORATION   
40317      2023  98983V10                             ZUORA, INC.   

                   sector                       value  \
fieldname                 AnalyticCO2Estimati

In [4]:
# Drop the upper level of the column index
fieldnames = [col[1] for col in pivot.columns if col[0] == 'value']
print(fieldnames)

pivot = pivot.droplevel(0, axis=1)

columnnames = ['year', 'cusip', 'comname', 'sector'] + fieldnames
pivot.columns = columnnames

print(pivot)

['AnalyticCO2EstimationMethod', 'AnalyticEstimatesCO2EquivalentsEmissionTotal']
       year     cusip                                 comname          sector  \
0      2002  00105510                      AFLAC INCORPORATED      FinanceIns   
1      2002  00184A10                        TIME WARNER INC.     Information   
2      2002  00185310                        Clearfield, Inc.   Manufacturing   
3      2002  00195750                              AT&T CORP.  DidNotIdentify   
4      2002  00282410                     ABBOTT LABORATORIES   Manufacturing   
...     ...       ...                                     ...             ...   
40313  2023  98980G10                           ZSCALER, INC.  DidNotIdentify   
40314  2023  98980L10               ZOOM COMMUNICATIONS, INC.  DidNotIdentify   
40315  2023  98981710                             ZUMIEZ INC.  DidNotIdentify   
40316  2023  98983L10  ZURN ELKAY WATER SOLUTIONS CORPORATION   Manufacturing   
40317  2023  98983V10        

# Getting Total Assets Data from Compustat

In [5]:
# Save all unique CUSIPs in the pivoted table above as a .txt file
unique_cusips = pivot['cusip'].unique()
np.savetxt('refinitivcusips_1_30_25.txt', unique_cusips, delimiter='\n', fmt='%s')

Use CUSIP convertor https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/tools/cusip-converter/ to convert to 9 digits (so it matches Compustat's).

Pull data from Compustat using list of CUSIPs above: **We need the data point at (Total Assets).**

CRSP/Compustat merged dataset in WRDS (https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/crspcompustat-merged/fundamentals-annual-with-prices/). You can get annual fundamentals, since they are only published once a year. Leave everything as is, and in the Compustat Variables look for “Assets - Total (at)”, while in the CRSP ones select the variables that you need to merge the datasets (either CUSIP, TICKER, PERMNO). This should match the ones you have already! 

In [6]:
# 'compustat_1_30_25.csv' is the data file containing at (Total Assets) downloaded directly from CRSP
compustat = pd.read_csv('compustat_1_30_25.csv', header=0)
print(compustat)

       gvkey    datadate indfmt consol popsrc datafmt      cusip curcd  \
0      39527  2022-12-31   INDL      C      D     STD  00032Q104   USD   
1      39527  2023-12-31   INDL      C      D     STD  00032Q104   USD   
2      21542  2002-12-31   INDL      C      D     STD  000360206   USD   
3      21542  2003-12-31   INDL      C      D     STD  000360206   USD   
4      21542  2004-12-31   INDL      C      D     STD  000360206   USD   
...      ...         ...    ...    ...    ...     ...        ...   ...   
56551  25128  2018-12-31   INDL      C      D     STD  98986X109   USD   
56552  25128  2019-12-31   INDL      C      D     STD  98986X109   USD   
56553  25128  2020-12-31   INDL      C      D     STD  98986X109   USD   
56554  25128  2021-12-31   INDL      C      D     STD  98986X109   USD   
56555  25128  2022-12-31   INDL      C      D     STD  98986X109   USD   

            at costat LINKTYPE  LPERMNO    MthCalDt  
0      184.237      A       LC    17869  2022-12-30  
1  

In [7]:
# Keep only relevant columns and rename to more familiar/intuitive column names
relevant = compustat.copy()[['cusip', 'datadate', 'at']]
relevant.rename(columns = {'at':'TotalAssets'}, inplace = True)
relevant.rename(columns = {'datadate':'date'}, inplace = True)
print(relevant)

           cusip        date  TotalAssets
0      00032Q104  2022-12-31      184.237
1      00032Q104  2023-12-31      132.423
2      000360206  2002-12-31       91.713
3      000360206  2003-12-31      102.085
4      000360206  2004-12-31      105.227
...          ...         ...          ...
56551  98986X109  2018-12-31       67.327
56552  98986X109  2019-12-31       87.765
56553  98986X109  2020-12-31       74.057
56554  98986X109  2021-12-31       81.172
56555  98986X109  2022-12-31       55.521

[56556 rows x 3 columns]


In [8]:
# Convert into relevant data formats
relevant['TotalAssets'] = relevant['TotalAssets'].astype(float)
relevant['date'] = pd.to_datetime(relevant['date'])
relevant['year'] = relevant['date'].dt.year
relevant['year'] = relevant['year'].astype(int)
relevant.drop(['date'], axis=1, inplace=True)

In [9]:
# Remove the last character from each string in 'cusip' to get back to 8-digit CUSIPs
relevant['cusip'] = relevant['cusip'].str[:-1]
print(relevant)

          cusip  TotalAssets  year
0      00032Q10      184.237  2022
1      00032Q10      132.423  2023
2      00036020       91.713  2002
3      00036020      102.085  2003
4      00036020      105.227  2004
...         ...          ...   ...
56551  98986X10       67.327  2018
56552  98986X10       87.765  2019
56553  98986X10       74.057  2020
56554  98986X10       81.172  2021
56555  98986X10       55.521  2022

[56556 rows x 3 columns]


In [10]:
# Confirming that all CUSIPs only 8 digits (so CUSIP digit length is consistent across different data sources)
relevantcopy = relevant.copy()
relevantcopy['cusipDigits'] = relevantcopy['cusip'].str.len()
print(relevantcopy['cusipDigits'].unique())

[8]


# Merging the Datasets

In [11]:
# Merge original data containing year, cusip, comname, sector and the CO2 columns
    # with the Compustat total assets data (merging on year and cusip!)
merged = pd.merge(pivot, relevant, on=['year', 'cusip'])
print(merged)

       year     cusip                                 comname          sector  \
0      2002  00105510                      AFLAC INCORPORATED      FinanceIns   
1      2002  00195750                              AT&T CORP.  DidNotIdentify   
2      2002  00282410                     ABBOTT LABORATORIES   Manufacturing   
3      2002  00724F10                              ADOBE INC.  DidNotIdentify   
4      2002  00790310            ADVANCED MICRO DEVICES, INC.   Manufacturing   
...     ...       ...                                     ...             ...   
30474  2023  98980G10                           ZSCALER, INC.  DidNotIdentify   
30475  2023  98980L10               ZOOM COMMUNICATIONS, INC.  DidNotIdentify   
30476  2023  98981710                             ZUMIEZ INC.  DidNotIdentify   
30477  2023  98983L10  ZURN ELKAY WATER SOLUTIONS CORPORATION   Manufacturing   
30478  2023  98983V10                             ZUORA, INC.      FinanceIns   

      AnalyticCO2Estimation

In [12]:
# Remove rows with missing data
merged.dropna(subset=['year'], inplace=True)
merged.dropna(subset=['cusip'], inplace=True)
merged.dropna(subset=['comname'], inplace=True)
merged.dropna(subset=['sector'], inplace=True)
merged.dropna(subset=['AnalyticCO2EstimationMethod'], inplace=True)
merged.dropna(subset=['AnalyticEstimatesCO2EquivalentsEmissionTotal'], inplace=True)
merged.dropna(subset=['TotalAssets'], inplace=True)

missing_counts = merged.isnull().sum()
print(missing_counts)

year                                            0
cusip                                           0
comname                                         0
sector                                          0
AnalyticCO2EstimationMethod                     0
AnalyticEstimatesCO2EquivalentsEmissionTotal    0
TotalAssets                                     0
dtype: int64


In [13]:
# Export this cleaned dataset (pivoted, with clean CUSIPs, Sectors, AND Total Assets) as a .csv and proceed to Step 3
merged.to_csv('RefinitivAndAssets_1_30_25.csv')